In [3]:
# --- Imports ---
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier as lgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,roc_auc_score

ModuleNotFoundError: No module named 'catboost'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
# --- Data Loading ---
train_df=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train_df.head()
train_df.describe()
train_df.info()
train_df['loan_paid_back'].unique()
train_df['gender'].unique()
train_df['marital_status'].unique()
train_df['education_level'].unique()
train_df['employment_status'].unique()
train_df['loan_purpose'].unique()
train_df['grade_subgrade'].unique()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [ ]:
# --- Preprocessing ---
# --- Doing One hot encoding of categorical Features---------
train_df=pd.get_dummies(train_df,columns=['gender','marital_status','education_level','employment_status','loan_purpose'],dtype=int)
test=pd.get_dummies(test,columns=['gender','marital_status','education_level','employment_status','loan_purpose'],dtype=int)

#---- Changing grades_subgrades columns to continous numbers-----
train_df['grade_subgrade'].unique()
grade=['A1','A2','A3','A4','A5',
       'B1','B2','B3','B4','B5',
       'C1','C2','C3','C4','C5',
       'D1','D2','D3','D4','D5',
       'E1','E2','E3','E4','E5',
       'F1','F2','F3','F4','F5']
grade_map={g: i for i,g in enumerate(grade)}
train_df['grade_subgrade']=train_df['grade_subgrade'].map(grade_map)
test['grade_subgrade']=test['grade_subgrade'].map(grade_map)

# Drop ID from training features
X = train_df.drop(columns=['id', 'loan_paid_back'])
y = train_df['loan_paid_back']

# Drop ID from test features (but store it for submission)
ids = test['id']
test = test.drop(columns=['id'])


In [ ]:
# ---Scaling Features---
scale=StandardScaler()
X=scale.fit_transform(X)

In [ ]:
# ----- Doing train/test split -------
x_train,x_,y_train,y_=train_test_split(X,Y,test_size=0.4,random_state=42)
x_cv,x_test,y_cv,y_test=train_test_split(x_,y_,test_size=0.5,random_state=42)

In [ ]:
# --- Align Features ---
test = test.reindex(columns=X.columns, fill_value=0)

In [ ]:
#----- DecisionTreeClassifier------
dec_model=DecisionTreeClassifier(max_depth=7)
dec_model.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=7)

In [ ]:
# Preidiction
dec_pred=dec_model.predict(x_test)
print(" Accuracy :",accuracy_score(y_test,dec_pred)*100)

print("Local DEC val AUC:", roc_auc_score(y_cv, dec_model.predict_proba(x_cv)[:,1]))


90.09082568035085


In [ ]:
#------- RandomForestClassifier------
rand_model=RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=-1,
    min_samples_leaf=40
    )
rand_model.fit(x_train,y_train)

RandomForestClassifier(max_depth=12, min_samples_leaf=40, n_estimators=300,
                       n_jobs=-1, random_state=42)

In [ ]:
rand_pred=rand_model.predict(x_test)
print(" Accuracy :",accuracy_score(y_test,rand_pred)*100)
print("Local RAF val AUC:", roc_auc_score(y_cv, rand_model.predict_proba(x_cv)[:,1]))

89.99739055042551

In [ ]:
# --- XGBClassifier------
xboost_model=XGBClassifier(
    n_estimators=1000,
    max_depth=12,
    random_state=42,
    learning_rate=0.11,
    min_data_in_leaf=20,
    reg_lambda=2,
    tree_method='hist'
)
xboost_model.fit(x_train,y_train)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:10:39] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "min_data_in_leaf" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.11, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=None, min_data_in_leaf=20,
              missing=nan, monotone_constraints=None, multi_strategy=None,
              n_estimators=1000, n_jobs=None, ...)

In [ ]:
xboost_pred=xboost_model.predict(x_test)
print("Accuracy :",accuracy_score(y_test,xboost_pred)*100)
print("Local XGB val AUC:", roc_auc_score(y_cv, xboost_model.predict_proba(x_cv)[:,1]))

89.91321475769998

In [ ]:
# --- LightBoostClassifier---
lightboost_model=lgb(
    n_estimators=900,
    max_depth=12,
    random_state=42,
    learning_rate=0.23,
    min_data_in_leaf=15,
    reg_lambda=2,
    tree_method='hist'

)
lightboost_model.fit(x_train,y_train)

[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Info] Number of positive: 284959, number of negative: 71437
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034671 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1354
[LightGBM] [Info] Number of data points in the train set: 356396, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.799557 -> initscore=1.383529
[LightGBM] [Info] Start training from score 1.383529


LGBMClassifier(learning_rate=0.23, max_depth=12, min_data_in_leaf=15,
               n_estimators=900, random_state=42, reg_lambda=2,
               tree_method='hist')

In [ ]:
lgb_pred=lightboost_model.predict(x_test)
print("Accuracy :",accuracy_score(y_test,lgb_pred)*100)
print("Local LGB val AUC:", roc_auc_score(y_cv, lightboost_model.predict_proba(x_cv)[:,1]))

[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15


90.40059259758078

In [ ]:
# ----CatBoostClassifier------
catboost_model=CatBoostClassifier(
    n_estimators=1000,
    max_depth=11,
    random_state=42,
    learning_rate=0.2,
    min_data_in_leaf=15,
    reg_lambda=2

)
catboost_model.fit(x_train,y_train)

0:	learn: 0.4928031	total: 181ms	remaining: 3m
1:	learn: 0.3807208	total: 417ms	remaining: 3m 28s
2:	learn: 0.3264953	total: 653ms	remaining: 3m 37s
3:	learn: 0.2967010	total: 907ms	remaining: 3m 45s
4:	learn: 0.2798928	total: 1.15s	remaining: 3m 48s
5:	learn: 0.2705413	total: 1.38s	remaining: 3m 47s
6:	learn: 0.2650008	total: 1.6s	remaining: 3m 47s
7:	learn: 0.2614030	total: 1.83s	remaining: 3m 47s
8:	learn: 0.2593219	total: 2.09s	remaining: 3m 49s
9:	learn: 0.2573542	total: 2.32s	remaining: 3m 49s
10:	learn: 0.2561407	total: 2.55s	remaining: 3m 49s
11:	learn: 0.2553739	total: 2.79s	remaining: 3m 49s
12:	learn: 0.2544669	total: 3.06s	remaining: 3m 52s
13:	learn: 0.2538121	total: 3.29s	remaining: 3m 51s
14:	learn: 0.2533327	total: 3.52s	remaining: 3m 51s
15:	learn: 0.2527004	total: 3.76s	remaining: 3m 51s
16:	learn: 0.2520237	total: 4.03s	remaining: 3m 53s
17:	learn: 0.2516525	total: 4.27s	remaining: 3m 53s
18:	learn: 0.2512135	total: 4.51s	remaining: 3m 52s
19:	learn: 0.2508622	total:

In [ ]:
catboost_pred=catboost_model.predict(x_test)
print("Accuracy :",accuracy_score(y_test,catboost_pred)*100)

print("Local CATB val AUC:", roc_auc_score(y_cv, catboost_model.predict_proba(x_cv)[:,1]))

90.05968063704239

In [ ]:
# Decission tree classifier
dec_pred=dec_model.predict_proba(test)[:, 1]
dec_sub=pd.DataFrame({'id':ids['id'],'loan_paid_back':dec_pred})
dec_sub.to_csv('submission_dec.csv',index=False)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [ ]:
# -- Saving submissions files--
# Random forest
rand_pred=rand_model.predict_proba(test)[:, 1]
rand_sub=pd.DataFrame({'id':ids['id'],'loan_paid_back':rand_pred})
rand_sub.to_csv('submission_rand.csv',index=False)

In [ ]:
# XboostClassifier
xgb_pred=xboost_model.predict_proba(test)[:, 1]
xgb_sub=pd.DataFrame({'id':ids['id'],'loan_paid_back':xgb_pred})
xgb_sub.to_csv('submission_xgb.csv',index=False)

In [ ]:
# Lightboost Clasisifier
lgb_pred=lightboost_model.predict_proba(test)[:, 1]
lgb_sub=pd.DataFrame({'id':ids['id'],'loan_paid_back':lgb_pred})
lgb_sub.to_csv('submission_lgb.csv',index=False)

[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15


In [ ]:
# Catboost Classifier
catb_pred=catboost_model.predict_proba(test)[:, 1]
catb_sub=pd.DataFrame({'id':ids['id'],'loan_paid_back':catb_pred})
catb_sub.to_csv('submission_catb.csv',index=False)